# Investigation of Commuter Routes from Bay Area Bike Share Published Data

In [1]:
%matplotlib inline

import matplotlib
import numpy as np
# from scipy import stats
# import scipy
import math
import matplotlib.pyplot as plt
import pandas as pd
from glob import glob
import datetime

import seaborn as sns
# sns.set()
sns.set_style('whitegrid')
sns.set_context("poster")

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

font = {'size'   : 50}
matplotlib.rc('font', **font)

TITLE_FONT_SIZE = 25
LABEL_FONT_SIZE = 15
TICK_FONT_SIZE  = 15

day_labels = ['MON','TUE','WED','THU','FRI','SAT','SUN']
day_labels_full = ['MONDAY','TUESDAY','WEDNESDAY','THURSDAY','FRIDAY','SATURDAY','SUNDAY']
month_labels = ['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC']

sub_color = 'b'
sub_color_alt = 'm'
cust_color='r'
cust_color_alt='y'

commuter_color='g'
commuter_color_alt='#1daf1d'

commuter_am = '#ea54d9'     #OrRd
commuter_am_alt = '#9b8460' #OrRd_r

commuter_pm = '#b97ccc'     #PuRd
commuter_pm_alt = '#f4ad3a' #PuRd_r

FIG_SIZE = (15,6)
GRID_DIMS = 15

# Load Morning and Evening Commuter Trips Data

In [3]:
print('[%s] Loading Morning Commute Trips Data...' % datetime.datetime.now().time())

morning_commutes = pd.DataFrame()
trip_data_file = '../../clean_data/bayareabikeshare/trip_data_morning_commutes.csv'

# Chunk Settings
chunks = []
chunk_counter = 1
chunksize = 10000
num_chunks = math.ceil(sum(1 for row in open(trip_data_file, 'r'))/chunksize)

# import file in chunks
for chunk in pd.read_csv(trip_data_file, chunksize=chunksize, iterator=True, index_col=0, parse_dates=['start_date', 'end_date', 'forecast_time']):
    
    # append chunk to chunks list
    chunks.append(chunk)

    if chunk_counter == 1 or chunk_counter % math.ceil(num_chunks/10) == 0 or chunk_counter == num_chunks:
        print('\t\t[%s] finished chunk %s of %s' % (datetime.datetime.now().time(), chunk_counter, num_chunks))
    chunk_counter += 1

morning_commutes = pd.concat(chunks)
morning_commutes.user_type = morning_commutes.user_type.astype('category')
print('[%s] Complete!' % datetime.datetime.now().time())


print('[%s] Loading Evening Commute Trips Data...' % datetime.datetime.now().time())

evening_commutes = pd.DataFrame()
trip_data_file = '../../clean_data/bayareabikeshare/trip_data_evening_commutes.csv'

# Chunk Settings
chunks = []
chunk_counter = 1
chunksize = 10000
num_chunks = math.ceil(sum(1 for row in open(trip_data_file, 'r'))/chunksize)

# import file in chunks
for chunk in pd.read_csv(trip_data_file, chunksize=chunksize, iterator=True, index_col=0, parse_dates=['start_date', 'end_date', 'forecast_time']):
    
    # append chunk to chunks list
    chunks.append(chunk)

    if chunk_counter == 1 or chunk_counter % math.ceil(num_chunks/10) == 0 or chunk_counter == num_chunks:
        print('\t\t[%s] finished chunk %s of %s' % (datetime.datetime.now().time(), chunk_counter, num_chunks))
    chunk_counter += 1

evening_commutes = pd.concat(chunks)
evening_commutes.user_type = evening_commutes.user_type.astype('category')
print('[%s] Complete!' % datetime.datetime.now().time())

[16:00:50.687551] Loading Morning Commute Trips Data...
		[16:00:52.521879] finished chunk 1 of 26
		[16:00:53.158991] finished chunk 3 of 26
		[16:00:54.031925] finished chunk 6 of 26
		[16:00:54.899077] finished chunk 9 of 26
		[16:00:56.207334] finished chunk 12 of 26
		[16:00:57.140047] finished chunk 15 of 26
		[16:00:58.047824] finished chunk 18 of 26
		[16:00:58.941235] finished chunk 21 of 26
		[16:00:59.782350] finished chunk 24 of 26
		[16:01:00.234218] finished chunk 26 of 26
[16:01:00.815331] Complete!
[16:01:00.815621] Loading Evening Commute Trips Data...
		[16:01:01.686540] finished chunk 1 of 24
		[16:01:02.290223] finished chunk 3 of 24
		[16:01:03.382758] finished chunk 6 of 24
		[16:01:04.393977] finished chunk 9 of 24
		[16:01:05.445190] finished chunk 12 of 24
		[16:01:06.325743] finished chunk 15 of 24
		[16:01:07.031230] finished chunk 18 of 24
		[16:01:07.703187] finished chunk 21 of 24
		[16:01:08.403604] finished chunk 24 of 24
[16:01:08.837979] Complete!


## Load Station Data

In [4]:
stations = pd.read_csv('../../clean_data/bayareabikeshare/station_data_cleaned.csv', 
                       index_col=0, parse_dates=['first_service_date', 'last_service_date'])

stations = stations[stations.region == 'San Francisco'].copy()
stations.reset_index(inplace=True, drop=True)

# station 73 was expanded, we dont care about that right now, we just want days open.
# copy last_service_date from second entry to first, need total days
stations.iloc[29,7] = stations.iloc[30,7]
# add days in sevice to second entry to first entry 
stations.iloc[29,9] += stations.iloc[30,9]
# drop second entry
stations.drop_duplicates(subset=['station_id'], inplace=True)

stations.reset_index(inplace=True, drop=True)
stations.head()

,station_id,station_name,lat,long,dock_count,region,first_service_date,last_service_date,zip_code,days_in_service,elevation_meters,elevation_feet
0,39,Powell Street BART,37.783871,-122.408433,19,San Francisco,2013-08-29,2016-08-31,94107,1098,10.338920,33.920340
1,41,Clay at Battery,37.795001,-122.399970,15,San Francisco,2013-08-29,2016-08-31,94107,1098,3.082564,10.113399
2,42,Davis at Jackson,37.797280,-122.398436,15,San Francisco,2013-08-29,2016-08-31,94107,1098,2.519689,8.266697
3,45,Commercial at Montgomery,37.794231,-122.402923,15,San Francisco,2013-08-29,2016-08-31,94107,1098,4.575804,15.012480
4,46,Washington at Kearney,37.795425,-122.404767,15,San Francisco,2013-08-29,2016-08-31,94107,1098,10.850125,35.597524


# Elevation Change on Route

In [5]:
def make_elevation_grid(stations=None, units='feet'):
    
    stations_df = stations.copy()
    stations_df.set_index('station_id', inplace=True, drop=True)

    for col_name in stations_df.columns:
        if 'elevation' not in col_name:
            stations_df.drop(col_name, axis=1, inplace=True)
    
    
    units = 'elevation_%s' % units.lower()
    
    units_id = list(stations_df.columns).index(units)
    
    index   = [x for x in stations_df.index]
    index = index[::-1]
    columns = [x for x in stations_df.index]
    
    grid = pd.DataFrame(index=index, columns=columns)

    # create grid of trips from start to end terminal counts
    for sid in stations_df.index:
        sid = list(stations_df.index).index(sid)
        for eid in stations_df.index:
            eid = list(stations_df.index).index(eid)
            sid_elevation = stations_df.iloc[sid, list(stations_df.columns).index(units)] 
            eid_elevation = stations_df.iloc[eid, list(stations_df.columns).index(units)] 
            
            elevation_diff = sid_elevation - eid_elevation

            grid.loc[sid, eid] = elevation_diff

    grid = grid.iloc[::-1]
    grid.dropna(how='all', axis=0, inplace=True)
    grid.dropna(how='all', axis=1, inplace=True)
    grid.fillna(0, inplace=True)
    
    grid.index = index
    grid.columns = columns
    
    return grid

In [6]:
station_elevation_grid = make_elevation_grid(stations=stations, units='feet')
for col in station_elevation_grid.columns:
    station_elevation_grid[col] = station_elevation_grid[col].astype('int')
    
mask = station_elevation_grid == 0
fig, ax = plt.subplots(figsize=(GRID_DIMS*1.5, GRID_DIMS))

grid_max = station_elevation_grid.max().max()
grid_min = station_elevation_grid.min().min()

cbar_ticks = [int(x) for x in np.arange(grid_min, grid_max+1, grid_max/5)]

sns.heatmap(data=station_elevation_grid, linecolor='grey', linewidths=.5, square=True, cmap=sns.diverging_palette(220, 20, n=15), 
            mask=mask, ax=ax, cbar_kws={"shrink": .75, "ticks":cbar_ticks}, cbar=True, annot=False, fmt="d", annot_kws={"size": 10})

ax.set_xlabel('Start Station', size=LABEL_FONT_SIZE, weight='bold')
ax.set_ylabel('End Station', size=LABEL_FONT_SIZE, weight='bold')

title = 'Elevation Change Between Stations (feet)'
ax.set_title(title, size=TITLE_FONT_SIZE, weight='bold')
# plt.show()
plt.savefig('../../charts/station_elevation_difference_feet.png')
plt.close()


In [7]:
station_elevation_grid = make_elevation_grid(stations=stations, units='meters')
for col in station_elevation_grid.columns:
    station_elevation_grid[col] = station_elevation_grid[col].astype('int')
    
mask = station_elevation_grid == 0
fig, ax = plt.subplots(figsize=(GRID_DIMS*1.5, GRID_DIMS))

grid_max = station_elevation_grid.max().max()
grid_min = station_elevation_grid.min().min()

cbar_ticks = [int(x) for x in np.arange(grid_min, grid_max+1, grid_max/5)]

sns.heatmap(data=station_elevation_grid, linecolor='grey', linewidths=.5, square=True, cmap=sns.diverging_palette(220, 20, n=15), 
            mask=mask, ax=ax, cbar_kws={"shrink": .75, "ticks":cbar_ticks}, cbar=True, annot=False, fmt="d", annot_kws={"size": 10})

ax.set_xlabel('Start Station', size=LABEL_FONT_SIZE, weight='bold')
ax.set_ylabel('End Station', size=LABEL_FONT_SIZE, weight='bold')

title = 'Elevation Change Between Stations (meters)'
ax.set_title(title, size=TITLE_FONT_SIZE, weight='bold')
# plt.show()
plt.savefig('../../charts/station_elevation_difference_meters.png')
plt.close()



# Morning Commuter Trips from Station 73

In [8]:
station_73_morning_commmutes = morning_commutes[morning_commutes.start_station_id == 73].copy()

In [9]:
destinations = station_73_morning_commmutes.groupby(station_73_morning_commmutes.end_station_id).count()['trip_id']
destinations = destinations.to_frame()
destinations.columns = ['trip_count']

plt.subplots(figsize=FIG_SIZE)
ax = sns.barplot(x=destinations.index, y=destinations.trip_count, color=commuter_am, alpha=0.35)
# ax.legend(loc=1, frameon=True)
ax.set_title('Station 73 Morning Commuter Traffic', size=TITLE_FONT_SIZE, weight='bold')
ax.set_xlabel('Destination Station ID', size=LABEL_FONT_SIZE)
ax.set_ylabel('Traffic Volume', size=LABEL_FONT_SIZE)
# plt.show()
plt.savefig('../../charts/station_trends/station_73_morning_commuter_destinations.png')
plt.close()


# Caltrain Arrival and Departure Times

In [10]:
southbound_caltrain_departure_times = {'6:03' : int(6 * 60 + 3), '6:38' : int(6 * 60 + 38),
                                    '6:47' : int(6 * 60 + 47), '7:24' : int(7 * 60 + 24),
                                    '7:08' : int(7 * 60 + 8), '7:57' : int(7 * 60 + 57),
                                    '7:51' : int(7 * 60 + 51), '8:07' : int(8 * 60 + 7),
                                    '8:24' : int(8 * 60 + 24), '8:11' : int(8 * 60 + 11),
                                    '8:58' : int(8 * 60 + 58), '8:53' : int(8 * 60 + 53),
                                    '9:07' : int(9 * 60 + 7), '9:29' : int(9 * 60 + 29),
                                    '9:11' : int(9 * 60 + 11), '9:52' : int(9 * 60 + 52),
                                    '10:09' : int(10 * 60 + 9), '10:50' : int(10 * 60 + 50),
                                    '11:17' : int(11 * 60 + 17), '11:48' : int(11 * 60 + 48),
                                    '12:48' : int(12 * 60 + 48), '13:48' : int(13 * 60 + 48),
                                    '14:48' : int(14 * 60 + 48), '15:48' : int(15 * 60 + 48),
                                    '15:50' : int(15 * 60 + 50), '16:50' : int(16 * 60 + 50),
                                    '17:02' : int(17 * 60 + 2), '17:36' : int(17 * 60 + 36),
                                    '17:31' : int(17 * 60 + 31), '17:42' : int(17 * 60 + 42),
                                    '18:06' : int(18 * 60 + 6), '17:51' : int(17 * 60 + 51),
                                    '18:33' : int(18 * 60 + 33), '18:27' : int(18 * 60 + 27),
                                    '18:42' : int(18 * 60 + 42), '19:06' : int(19 * 60 + 6),
                                    '18:51' : int(18 * 60 + 51), '19:33' : int(19 * 60 + 33),
                                    '19:27' : int(19 * 60 + 27), '19:49' : int(19 * 60 + 49),
                                    '20:02' : int(20 * 60 + 2), '20:42' : int(20 * 60 + 42),
                                    '21:20' : int(21 * 60 + 20), '22:20' : int(22 * 60 + 20),
                                    '23:20' : int(23 * 60 + 20)}



northbound_caltrain_arrival_times = {'4:55' : int(4 * 60) + 55, '5:25' : int(5 * 60) + 25, '6:05' : int(6 * 60) + 5,
                                    '6:15' : int(6 * 60) + 15, '6:35' : int(6 * 60) + 35, '6:45' : int(6 * 60) + 45,
                                    '6:59' : int(6 * 60) + 59, '7:05' : int(7 * 60) + 5, '7:15' : int(7 * 60) + 15,
                                    '7:35' : int(7 * 60) + 35, '7:45' : int(7 * 60) + 45, '7:59' : int(7 * 60) + 59,
                                    '8:05' : int(8 * 60) + 5, '8:15' : int(8 * 60) + 15, '8:35' : int(8 * 60) + 35,
                                    '8:45' : int(8 * 60) + 45, '9:00' : int(9 * 60) + 0, '9:45' : int(9 * 60) + 45,
                                    '10:00' : int(10 * 60) + 0, '11:00' : int(11 * 60) + 0, '12:00' : int(12 * 60) + 0,
                                    '13:00' : int(13 * 60) + 0, '14:00' : int(14 * 60) + 0, '14:43' : int(14 * 60) + 43,
                                    '15:00' : int(15 * 60) + 0, '15:34' : int(15 * 60) + 34, '16:12' : int(16 * 60) + 12,
                                    '16:23' : int(16 * 60) + 23, '16:32' : int(16 * 60) + 32, '16:38' : int(16 * 60) + 38,
                                    '16:58' : int(16 * 60) + 58, '17:16' : int(17 * 60) + 16, '17:27' : int(17 * 60) + 27,
                                    '17:32' : int(17 * 60) + 32, '17:38' : int(17 * 60) + 38, '17:58' : int(17 * 60) + 58,
                                    '18:16' : int(18 * 60) + 16, '18:23' : int(18 * 60) + 23, '18:32' : int(18 * 60) + 32,
                                    '18:38' : int(18 * 60) + 38, '18:58' : int(18 * 60) + 58, '19:30' : int(19 * 60) + 30,
                                    '20:30' : int(20 * 60) + 30, '21:30' : int(21 * 60) + 30, '22:40' : int(22 * 60) + 40}

# Caltrain Super Station
<p> Inspect usage trends at the SuperStation, the combination of stations 69 and 70</p>

In [11]:
super_stations = [69,70]

In [12]:
ss_morning_commutes = morning_commutes[(morning_commutes.start_station_id.isin(super_stations)) 
                                       | (morning_commutes.end_station_id.isin(super_stations))].copy()
ss_evening_commutes = evening_commutes[(evening_commutes.start_station_id.isin(super_stations)) 
                                       | (evening_commutes.end_station_id.isin(super_stations))].copy()

In [13]:
def commute_hours_fill(df=None, dummy_df=None, group_date='start_date'):
    tmp = df.groupby([df[group_date].dt.hour, df[group_date].dt.minute]).count()['trip_id'].to_frame()
    tmp.index.names = ['hour', 'minute']
    tmp.reset_index(inplace=True, drop=False)
    tmp['dummy'] = tmp.hour * 60 + tmp.minute
    tmp.drop(['hour', 'minute'], axis=1, inplace=True)
    tmp.set_index('dummy', inplace=True, drop=True)
    tmp = tmp.merge(dummy_df, left_index=True, right_index=True, how='right')
    tmp.set_index('dummy', inplace=True, drop=True)
    tmp.index.names = ['minuteofcommute']
    tmp.fillna(0, inplace=True)
    
    return tmp

am_commute_hour_begin  = 7
am_commute_hour_finish = 10
pm_commute_hour_begin  = 16
pm_commute_hour_finish = 19

dummy_morning_commute = pd.DataFrame(index=[x for x in range(am_commute_hour_begin*60, am_commute_hour_finish*60, 1)], columns=['dummy'])
dummy_morning_commute['dummy'] = dummy_morning_commute.index.to_series()

dummy_evening_commute = pd.DataFrame(index=[x for x in range(pm_commute_hour_begin*60, pm_commute_hour_finish*60, 1)], columns=['dummy'])
dummy_evening_commute['dummy'] = dummy_evening_commute.index.to_series()

In [14]:
# MORNING COMMUTERS - Inbound Trains
trips_starting = commute_hours_fill(df=ss_morning_commutes, dummy_df=dummy_morning_commute, group_date='start_date')
starting_x_ticks = sorted(trips_starting.index.unique())

x_ticks = [x*60 for x in range(0, am_commute_hour_finish-am_commute_hour_begin+1, 1)]
x_markers = [x*60 for x in range(1, len(x_ticks))]

x_tick_labels = [str(x)+':00' for x in range(am_commute_hour_begin, am_commute_hour_finish+1, 1)]

plt.subplots(figsize=(15,4))
ax = sns.barplot(x = starting_x_ticks , y = 'trip_id', data=trips_starting, color=commuter_am, alpha = 0.5, label='Trips Starting')

ax.set_title('Morning Caltrain Super Station Outbound Bike Trips and Inbound Trains', size=TITLE_FONT_SIZE*0.75, weight='bold')

ax.set_xlabel('', size=TICK_FONT_SIZE)
ax.set_xticks(x_ticks)
ax.set_xticklabels(x_tick_labels, size=TICK_FONT_SIZE)
for x in x_markers:
    ax.axvline(x=x, linestyle=':', alpha=0.25, color='k')

    
# plot caltrain arrivals into SF
for k, v in northbound_caltrain_arrival_times.items():
    if v > am_commute_hour_begin*60 and v < am_commute_hour_finish*60:
        ax.axvline(x=v-am_commute_hour_begin*60, linestyle='-', alpha=0.5, color=commuter_am_alt)
        
ax.set_ylabel('Trip Count', size=LABEL_FONT_SIZE)

# ax.legend(loc=1, frameon=True)
# plt.show()
plt.savefig('../../charts/user_trends/superstation_morning_outbound_trips.png')
plt.close()



In [15]:
# MORNING COMMUTERS - Outbound Trains
trips_ending = commute_hours_fill(df=ss_morning_commutes, dummy_df=dummy_morning_commute, group_date='end_date')

ending_x_ticks = sorted(trips_ending.index.unique())

x_ticks = [x*60 for x in range(0, am_commute_hour_finish-am_commute_hour_begin+1, 1)]
x_markers = [x*60 for x in range(1, len(x_ticks))]

x_tick_labels = [str(x)+':00' for x in range(am_commute_hour_begin, am_commute_hour_finish+1, 1)]

plt.subplots(figsize=(15,4))
ax = sns.barplot(x = ending_x_ticks , y = 'trip_id', data=trips_ending, color=commuter_am_alt, alpha = 0.5, label='Trips Ending')

ax.set_title('Morning Caltrain Super Station Inbound Bike Trips and Outbound Trains', size=TITLE_FONT_SIZE*0.75, weight='bold')

ax.set_xlabel('', size=TICK_FONT_SIZE)
ax.set_xticks(x_ticks)
ax.set_xticklabels(x_tick_labels, size=TICK_FONT_SIZE)
for x in x_markers:
    ax.axvline(x=x, linestyle=':', alpha=0.25, color='k')

    
# plot caltrain departures from SF
for k, v in southbound_caltrain_departure_times.items():
    if v > am_commute_hour_begin*60 and v < am_commute_hour_finish*60:
        ax.axvline(x=v-am_commute_hour_begin*60, linestyle='-', alpha=0.5, color=commuter_am)
        
ax.set_ylabel('Trip Count', size=LABEL_FONT_SIZE)

# ax.legend(loc=1, frameon=True)
# plt.show()
plt.savefig('../../charts/user_trends/superstation_morning_inbound_trips.png')
plt.close()

In [16]:
# EVENING COMMUTERS - Inbound Trains
trips_starting = commute_hours_fill(df=ss_evening_commutes, dummy_df=dummy_evening_commute, group_date='start_date')
starting_x_ticks = sorted(trips_starting.index.unique())

x_ticks = [x*60 for x in range(0, pm_commute_hour_finish-pm_commute_hour_begin+1, 1)]
x_markers = [x*60 for x in range(1, len(x_ticks))]

x_tick_labels = [str(x)+':00' for x in range(pm_commute_hour_begin, pm_commute_hour_finish+1, 1)]

plt.subplots(figsize=(15,4))
ax = sns.barplot(x = starting_x_ticks , y = 'trip_id', data=trips_starting, color=commuter_pm, alpha = 0.5, label='Trips Starting')

ax.set_title('Evening Caltrain Super Station Outbound Bike Trips and Inbound Trains', size=TITLE_FONT_SIZE*0.75, weight='bold')

ax.set_xlabel('', size=TICK_FONT_SIZE)
ax.set_xticks(x_ticks)
ax.set_xticklabels(x_tick_labels, size=TICK_FONT_SIZE)
for x in x_markers:
    ax.axvline(x=x, linestyle=':', alpha=0.25, color='k')

    
# plot caltrain arrivals into SF
for k, v in northbound_caltrain_arrival_times.items():
    if v > pm_commute_hour_begin*60 and v < pm_commute_hour_finish*60:
        ax.axvline(x=v-pm_commute_hour_begin*60, linestyle='-', alpha=0.5, color=commuter_pm_alt)
        
ax.set_ylabel('Trip Count', size=LABEL_FONT_SIZE)

# ax.legend(loc=1, frameon=True)
# plt.show()
plt.savefig('../../charts/user_trends/superstation_evening_outbound_trips.png')
plt.close()

In [17]:
# Evening COMMUTERS - Outbound Trains
trips_ending = commute_hours_fill(df=ss_evening_commutes, dummy_df=dummy_evening_commute, group_date='end_date')
ending_x_ticks = sorted(trips_ending.index.unique())

x_ticks = [x*60 for x in range(0, pm_commute_hour_finish-pm_commute_hour_begin+1, 1)]
x_markers = [x*60 for x in range(1, len(x_ticks))]

x_tick_labels = [str(x)+':00' for x in range(pm_commute_hour_begin, pm_commute_hour_finish+1, 1)]

plt.subplots(figsize=(15,4))
ax = sns.barplot(x = ending_x_ticks , y = 'trip_id', data=trips_ending, color=commuter_pm_alt, alpha = 0.5, label='Trips Ending')

ax.set_title('Evening Caltrain Super Station Inbound Bike Trips and Outbound Trains', size=TITLE_FONT_SIZE*0.75, weight='bold')

ax.set_xlabel('', size=TICK_FONT_SIZE)
ax.set_xticks(x_ticks)
ax.set_xticklabels(x_tick_labels, size=TICK_FONT_SIZE)
for x in x_markers:
    ax.axvline(x=x, linestyle=':', alpha=0.25, color='k')

    
# plot caltrain departures from SF
for k, v in southbound_caltrain_departure_times.items():
    if v > pm_commute_hour_begin*60 and v < pm_commute_hour_finish*60:
        ax.axvline(x=v-pm_commute_hour_begin*60, linestyle='-', alpha=0.5, color=commuter_pm)
        
ax.set_ylabel('Trip Count', size=LABEL_FONT_SIZE)

# ax.legend(loc=1, frameon=True)
# plt.show()
plt.savefig('../../charts/user_trends/superstation_evening_inbound_trips.png')
plt.close()